In [91]:
# Dependencies
import gmaps
import pandas as pd
import numpy as np
import requests
import json
from config import gkey
from geopy import geocoders
import time

# Google API Key
gmaps.configure(api_key=gkey)

In [92]:
# import csv with winery names and country names
# File to Load
wine_csv = "resources/red_and_white_combined.csv"

# Read wine data file and store into Pandas DataFrames
full_df = pd.read_csv(wine_csv)

In [93]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12430 entries, 0 to 12429
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             12430 non-null  object 
 1   Country          12430 non-null  object 
 2   Region           12430 non-null  object 
 3   Winery           12430 non-null  object 
 4   Rating           12430 non-null  float64
 5   NumberOfRatings  12430 non-null  int64  
 6   Price            12430 non-null  float64
 7   Year             12430 non-null  object 
 8   Type             12430 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 874.1+ KB


In [94]:
full_df.head()

,Name,Country,Region,Winery,Rating,NumberOfRatings,Price,Year,Type
0,Vermentino 2017,Italy,Toscana,Famiglia Castellani,3.8,25,5.65,2017,white
1,Ronco Broilo 2010,Italy,Colli Orientali del Friuli,Conte d'Attimis Maniago,4.3,25,44.90,2010,white
2,Weisser Schiefer s 2017,Austria,Südburgenland,Weinbau Uwe Schiefer,4.2,25,33.25,2017,white
3,Chardonnay 2018,Germany,Rheinhessen,Krämer - Straight,3.9,25,8.99,2018,white
4,Maganza Zibibbo 2018,Italy,Terre Siciliane,Luna Gaia,3.9,25,8.60,2018,white


In [73]:
# pulling columns needed for google API and adding them to new df
df = pd.DataFrame()
df["country"] = full_df["Country"].astype(str)
df["region"] = full_df["Region"].astype(str)
df["winery"] = full_df["Winery"].astype(str)
df["name"] = full_df["Name"].astype(str)
df["rating"] = full_df["Rating"]

In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12430 entries, 0 to 12429
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  12430 non-null  object 
 1   region   12430 non-null  object 
 2   winery   12430 non-null  object 
 3   name     12430 non-null  object 
 4   rating   12430 non-null  float64
dtypes: float64(1), object(4)
memory usage: 485.7+ KB


In [75]:
mask = df["rating"] >= 4.0
df2 = df[mask]
df2.head()

,country,region,winery,name,rating
1,Italy,Colli Orientali del Friuli,Conte d'Attimis Maniago,Ronco Broilo 2010,4.3
2,Austria,Südburgenland,Weinbau Uwe Schiefer,Weisser Schiefer s 2017,4.2
6,Germany,Nahe,Diel,Goldloch Riesling GG 2016,4.1
7,Germany,Pfalz,Von Winning,Kieselberg GG 2015,4.1
11,France,Alsace,Domaine Pfister,Berg Riesling 2018,4.2


In [81]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4717 entries, 1 to 12426
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  4717 non-null   object 
 1   region   4717 non-null   object 
 2   winery   4717 non-null   object 
 3   name     4717 non-null   object 
 4   rating   4717 non-null   float64
dtypes: float64(1), object(4)
memory usage: 221.1+ KB


In [82]:
# Code to test API pull
# test = df2.iloc[:100]
# test.info()

In [83]:
# LARGE API DATA PULL - UNCOMMENT TO RUN. BUT UNECESSARY SINCE DATA WAS ADDED TO A CSV TO PREVENT NEED TO REPULL FROM API.

rows = []

# assume null
lat = np.nan
lng = np.nan

for indx, row in df2.iterrows():
    # flat dictionary
    new_row = {} 
    
    # Target city
    text = f"{row.winery}"
    country = f"{row.country}"
    wine_name = f"{row.name}"
    serves_wine = True
    print(text)
    
    # Build the endpoint URL
    target_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={text}&country={country}&serves_wine={serves_wine}&key={gkey}"
    
    try:
        # Run a request to endpoint and convert result to json
        response = requests.get(target_url)
        if(response.status_code == 200):

            geo_data = response.json()

            # Extract latitude and longitude
            if (len(geo_data["results"]) > 0):
                lat = geo_data["results"][0]["geometry"]["location"]["lat"]
                lng = geo_data["results"][0]["geometry"]["location"]["lng"]

            else:
                lat = np.nan
                lng = np.nan
                print("no winery found")
        else:
            lat = np.nan
            lng = np.nan
            
    except Exception as e:
        print(e)

         # nuke it, everything null
        lat = np.nan
        lng = np.nan
                    
    # flatten data and append
    new_row["name"] = wine_name
    new_row["winery"] = text
    new_row["country"] = country
    new_row["latitude"] = lat
    new_row["longitude"] = lng
    
    rows.append(new_row)
    
    # let api breathe
    time.sleep(3)

Conte d'Attimis Maniago
Weinbau Uwe Schiefer
Diel
no winery found
Von Winning
Domaine Pfister
Egon Müller - Scharzhof
no winery found
Rings
no winery found
Rings
no winery found
Château de Schengen
Wittmann
Mezzacorona
no winery found
René Muré
no winery found
Finca Garabelos
no winery found
Dr. Heger
Ellermann-Spiegel
Pietracupa
Castel Sallegg
Cusumano
Suavia
Weingut Bründlmayer
Prinz von Hessen
Domaine de la Janasse
no winery found
Pascal Jolivet
no winery found
Acustic Celler
Château de Fosse-Sèche
Luddite
no winery found
Gauby
Château Gazin Rocquencourt
Castel Sallegg
Kühling-Gillot
F.X. Pichler
Saint Clair
Markus Molitor
no winery found
Karl Pfaffmann
Van Volxem
Vaccaro
Sandhi
no winery found
Georg Mosbacher
Joh. Jos. Prüm
Wagner-Stempel
Weingüter Wegeler
no winery found
Tiefenbrunner
no winery found
Von Winning
Villabella
Ziereisen
Roveglia
no winery found
Juliusspital
Korrell
Dr. Von Bassermann-Jordan
Ökonomierat Rebholz
Château de Saint Louans
Gut Hermannsberg
Lvnae
no winery f

Faiveley
no winery found
Cà dei Frati
Battenfeld Spanier
Kühling-Gillot
Knipser
no winery found
Domaine de la Solitude
Dr. Bürklin-Wolf
no winery found
Botani
Pichler-Krutzler
Cape of Good Hope
Gross
no winery found
Comte Lafond
no winery found
Prager
no winery found
Domaine des Ardoisieres
The Kings Series
no winery found
Prager
no winery found
Boekenhoutskloof
Von Winning
Markus Molitor
no winery found
Les Héritiers du Comte Lafon
Cape of Good Hope
Weiser-Künstler
Arabella
no winery found
Domaine William Fèvre
The Kings Series
no winery found
Kühling-Gillot
Iona
Saint Clair
Prinz Salm
Maximin Grünhaus
Errazuriz
no winery found
Château de Chamirey
Domaine Saumaize-Michelin
David Moret-Nomine
José Pariente
no winery found
Vincent Girardin
no winery found
Ornellaia
no winery found
Domaine de Chevalier
no winery found
Weingüter Wegeler
no winery found
Reinhard Knebel
no winery found
Bonneau du Martray
Schneider
no winery found
Bret Brothers & The Soufrandière
no winery found
Grosset
no w

no winery found
Abadia Retuerta
Robert Weil
no winery found
Gaja
Lungarotti
Domäne Wachau
Panizzi
no winery found
Livio Felluga
no winery found
Jean-Marc Brocard
no winery found
Faiveley
no winery found
Schneider
no winery found
Domaine de Beaurenard
Round Pond Estate
Von Winning
Château de Sancerre
The Kings Series
no winery found
Talamonti
Quintodecimo
Domaines Ott
no winery found
Tenuta Gorghi Tondi
Milan Nestarec
Stina
Anselmi
Elena Walch
Herdade dos Grous
M. Chapoutier
no winery found
Peter Jakob Kühn
Domaine du Closel - Château des Vaults
Librandi
no winery found
Billaud-Simon
Alphonse Mellot
Merry Edwards
Château de Beaucastel
Κτημα Τσελεπου (Ktima Tselepos)
Gaja
Markus Molitor
no winery found
Weingut Erich & Walter Polz
no winery found
Pazo Barrantes
Knewitz
Valentini
Van Volxem
Domaine Michel Niellon
Minuty
no winery found
Colterenzio (Schreckbichl)
no winery found
Vasse Felix
Joseph Drouhin
no winery found
Vitis Metamorfosis
M. Chapoutier
no winery found
Fernlands
Bodegas Par

Negro Angelo
Domaine Lafage
no winery found
Castello di Gabbiano
Borgogno
no winery found
Antinori
no winery found
Teso La Monja
Barone Montalto
no winery found
Château Léoville Las Cases
no winery found
Château l'Ecuyer
no winery found
Gabbas
no winery found
Bodegas Amaren
Zýmē
no winery found
Jean Leon
Aia Vecchia
Ciacci Piccolomini d'Aragona
Zenato
no winery found
Vallepicciola
De Toren
no winery found
Château Beychevelle
Clavel
Knipser
no winery found
Ottin - Viticulteur Encaveur
Louis Jadot
no winery found
Paul Jaboulet Aîné
no winery found
Domaine du Clos des Fées
Prunotto
no winery found
Campillo
no winery found
Château la Vieille Cure
Tenuta Il Poggione
Château Léoville Barton
Château Smith Haut-Lafitte
Catena
no winery found
Château Lafon-Rochet
Bolla
no winery found
Château Belair-Monange
Château Lascombes
Promontory
Conti Zecca
Chanson
no winery found
Borgogno
no winery found
Catena Zapata
Château Purcari
Cave de Tain
Domaine Serene
El Enemigo
no winery found
Benjamin de Rot

Faiveley
no winery found
Casa Ferreirinha
no winery found
Planeta
no winery found
Bouchard Finlayson
Mukhrani
Henschke
no winery found
Domaine Mathieu & Camille Lapierre
Pago de Los Capellanes
no winery found
Château Giscours
Château Léoville Barton
Alta Vista
no winery found
Torre dei Beati
Terra d'Alter
no winery found
Braida
no winery found
Ceretto
Château Branaire-Ducru
Muga
no winery found
Paul Hobbs
Luna Argenta
no winery found
San Marzano
Domaine Serene
Pietra Pura
Domaine Anderson
Braida
no winery found
Fitapreta
Tenuta Ulisse
Château La Couronne
Les Bertrand
Le Ragose
E. Guigal
no winery found
Biondi-Santi
no winery found
Bodegas San Román
Vigneti del Vulture
Poggio Antico
no winery found
Château Ducru-Beaucaillou
Casetta
no winery found
Tinto Pesquera
Baron de Ley
no winery found
Baigorri
Elio Grasso
Tenute Silvio Nardi
Drei Donà - Tenuta La Palazza
Álvaro Palacios
Ciacci Piccolomini d'Aragona
Château Phélan Ségur
Clos Puy Arnaud
Le Macioche
Tua Rita
Château Clinet
no winery 

...

Los Vascos
no winery found
Château Belair-Monange
Delas
no winery found
Didier Desvignes
Villa Sandi
no winery found
Paul Jaboulet Aîné
no winery found
Betz Family Winery
Paolo Cottini
d'Arenberg
Geoff Merrill
Poliziano
no winery found
Errazuriz
no winery found
Castelfeder
no winery found
Chiara Condello
no winery found
Bertani
no winery found
Polvanera
Gran Sello
no winery found
Costa Arente
Tua Rita
Lewis Cellars
La Cabotte
Rivetti Massimo
Pietroso
De'Ricci
Doolhof Wine Estate
Georges Vigouroux
Acustic Celler
Clos de l'Oratoire
Falesco
Château Haut-Brion
no winery found
Poggio Al Tesoro
Château Cantenac Brown
Planeta
no winery found
Gaja
EGO
no winery found
Château La Conseillante
Schola Sarmenti
Il Marroneto
Pascual Toso
no winery found
Poggio Landi
Epicuro
no winery found
Château Marquis de Terme
Galardi
Château Gruaud Larose
Château Léoville Barton
Orin Swift
no winery found
Nino Negri
no winery found
Spice Route
Esporão
no winery found
Morgenster Estate
Antinori
no winery found
C

no winery found
Villa Canestrari
no winery found
Château le Bedat
no winery found
Cascina Luisin
Anita Nittnaus Hans
Domaine Bousquet
Ses Talaioles
Marqués de Riscal
no winery found
Château Giscours
Portada
no winery found
Famille Perrin
no winery found
Biserno
no winery found
Marqués de Riscal
no winery found
Thymiopoulos
Two Hands
no winery found
Fontodi
no winery found
Montevertine
Neethlingshof Estate
Clos Mogador
no winery found
Catena Zapata
Valderiz
no winery found
Selvapiana
Schug
no winery found
Cottini
no winery found
Tedeschi
no winery found
Esterházy
Nicolis
Epicuro
no winery found
Emilio Moro
no winery found
St. Michael-Eppan
Menhir
no winery found
Mauro Molino
Tua Rita
Martoccia
Serpaia di Endrizzi
no winery found
Heribert Bayer
D. Mateos
no winery found
Boekenhoutskloof
Fattoria I Veroni
Bodegas Mas Alta
Spottswoode
Château Talbot
no winery found
Cantine Paradiso
Brazin
no winery found
Résonance
no winery found
Tenuta CastelGiocondo
Marqués de Murrieta
no winery found
Ch

Château Mouton Rothschild
no winery found
Château d'Issan
Taylors / Wakefield
no winery found
Château de la Commanderie
Château Pavie
Château Ducru-Beaucaillou
Château Rauzan-Gassies
Isole e Olena
Lenotti
Gulfi
Marqués de Griñon
Poggio Al Tesoro
Encierra
no winery found
Herdade Grande
La Spinetta
no winery found
E. Pira & Figli
no winery found
Borgogno
no winery found
Domaine de la Janasse
no winery found
Casa di Terra
Hudelot-Noëllat
Pranzegg
Terre del Marchesato
La Mondotte
no winery found
Drei Donà - Tenuta La Palazza
Clos Dubreuil
Hans Igler
Château Lacour Jacquet
Gulfi
Quinta da Gandara
Azienda Agricola Accornero
Château Meyney
Château Haut-Bailly
Dominus
no winery found
Errazuriz
no winery found
Niepoort
no winery found
Nino Negri
no winery found
Rust En Vrede
Schneider
no winery found
Zeni
no winery found
Vincent Girardin
no winery found
Lichtenberger & González
no winery found
Faiveley
no winery found
Krutzler
Herdade Grande
San Fabiano Calcinaia
Manincor
Delas
no winery found


Cantina Bolzano / Kellerei Bozen
Firriato
Paolo Manzone
Château Haut-Brion
no winery found
Zenato
no winery found
Gesellmann
Mezzacorona
no winery found
Selva Capuzza
Louis Jadot
no winery found
Mitolo
Château La Gaffelière
Vinha Maria
no winery found
Schola Sarmenti
Château Léoville Las Cases
no winery found
De Toren
no winery found
Gaja
d'Arenberg
Vega Sicilia
no winery found
M. Chapoutier
no winery found
Les Bertrand
Château Ducru-Beaucaillou
Vegalfaro
Delas
no winery found
Cantina Kaltern
Clos de l'Oratoire
Cascina Luisin
Château d'Esclans
no winery found
Borgogno
no winery found
Joseph Drouhin
no winery found
Boscarelli
Marimar Estate
no winery found
Tramin
Z1N
no winery found
Louis Latour
no winery found
Beringer
Lan
no winery found
La Rioja Alta
Imperial
no winery found
Allegrini
no winery found
Cielo e Terra
Bodegas Faustino
Penfolds
no winery found
The Prisoner
no winery found
Salentein
no winery found
Château Ducru-Beaucaillou
Bouchard Père & Fils
no winery found
Bruno Duchên

no winery found
Glaetzer
no winery found
Le Mortelle


In [84]:
rows

[{'name': '1',
  'winery': "Conte d'Attimis Maniago",
  'country': 'Italy',
  'latitude': 46.0096606,
  'longitude': 13.3416919},
 {'name': '2',
  'winery': 'Weinbau Uwe Schiefer',
  'country': 'Austria',
  'latitude': 47.2340461,
  'longitude': 16.3484524},
 {'name': '6',
  'winery': 'Diel',
  'country': 'Germany',
  'latitude': nan,
  'longitude': nan},
 {'name': '7',
  'winery': 'Von Winning',
  'country': 'Germany',
  'latitude': 49.4106226,
  'longitude': 8.1863799},
 {'name': '11',
  'winery': 'Domaine Pfister',
  'country': 'France',
  'latitude': 48.5854917,
  'longitude': 7.508338699999999},
 {'name': '13',
  'winery': 'Egon Müller - Scharzhof',
  'country': 'Germany',
  'latitude': nan,
  'longitude': nan},
 {'name': '18',
  'winery': 'Rings',
  'country': 'Germany',
  'latitude': nan,
  'longitude': nan},
 {'name': '28',
  'winery': 'Rings',
  'country': 'Germany',
  'latitude': nan,
  'longitude': nan},
 {'name': '35',
  'winery': 'Château de Schengen',
  'country': 'Luxemb

In [89]:
df_loc = pd.DataFrame(rows)
df_loc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4717 entries, 0 to 4716
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       4717 non-null   object 
 1   winery     4717 non-null   object 
 2   country    4717 non-null   object 
 3   latitude   2942 non-null   float64
 4   longitude  2942 non-null   float64
dtypes: float64(2), object(3)
memory usage: 184.4+ KB


In [90]:
df_loc_to_csv = dict(archive_name='lat_and_long.csv')  
df_loc.to_csv('lat_and_long.csv', index=False) 